In [1]:
""" GraspNet dataset processing.
    Author: chenxi-wang
"""

import os
import sys
import numpy as np
import scipy.io as scio
from PIL import Image

import torch
import collections.abc as container_abcs
from torch.utils.data import Dataset
from tqdm import tqdm
BASE_DIR = os.path.dirname('/home/po/TM5/graspnet-baseline/dataset/graspnet_dataset.py')
ROOT_DIR = BASE_DIR
sys.path.append(os.path.join('/home/po/TM5/graspnet-baseline', 'utils'))
from data_utils import CameraInfo, transform_point_cloud, create_point_cloud_from_depth_image,\
                            get_workspace_mask, remove_invisible_grasp_points
BASE_DIR

'/home/po/TM5/graspnet-baseline/dataset'

In [2]:
#graspnetAPI graspnet.py API interface Path
sys.path.append('/home/po/TM5/graspnetAPI/graspnetAPI')
sys.path.append('/home/po/TM5/graspnetAPI/graspnetAPI/utils')
from graspnetAPI.grasp import Grasp, GraspGroup, RectGrasp, RectGraspGroup, RECT_GRASP_ARRAY_LEN
from graspnetAPI.utils.utils import transform_points, parse_posevector
from graspnetAPI.utils.xmlhandler import xmlReader

TOTAL_SCENE_NUM = 25 #190  ori
GRASP_HEIGHT = 0.02

from pyquaternion import Quaternion

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


WARNING - 2021-11-25 01:10:58,372 - rigid_transformations - Failed to import ros dependencies in rigid_transforms.py
WARNING - 2021-11-25 01:10:58,373 - rigid_transformations - autolab_core not installed as catkin package, RigidTransform ros methods will be unavailable


In [3]:
class GraspNetDataset(Dataset):
    def __init__(self, root, valid_obj_idxs, grasp_labels, camera='realsense', split='train', num_points=10000,
                 remove_outlier=False, remove_invisible=True, augment=False, load_label=True):
        assert(num_points<=50000)
        self.root = root
        self.split = split
        self.num_points = num_points
        self.remove_outlier = remove_outlier
        self.remove_invisible = remove_invisible
        self.valid_obj_idxs = valid_obj_idxs
        self.grasp_labels = grasp_labels
        self.camera = camera
        self.augment = augment
        self.load_label = load_label
        self.collision_labels = {}
        self.template_grasp = np.zeros([20000,7])
        if split == 'train':
            self.sceneIds = list( range(100) )
        elif split == 'test':
            self.sceneIds = list( range(100,190) )
        elif split == 'test_seen':
            self.sceneIds = list( range(100,130) )
        elif split == 'test_similar':
            self.sceneIds = list( range(130,160) )
        elif split == 'test_novel':
            self.sceneIds = list( range(160,190) )
        self.sceneIds = ['scene_{}'.format(str(x).zfill(4)) for x in self.sceneIds]
        
        self.colorpath = []
        self.depthpath = []
        self.labelpath = []
        self.metapath = []
        self.scenename = []
        self.frameid = []
        for x in tqdm(self.sceneIds, desc = 'Loading data path and collision labels...'):
            for img_num in range(256):
                self.colorpath.append(os.path.join(root, 'scenes', x, camera, 'rgb', str(img_num).zfill(4)+'.png'))
                self.depthpath.append(os.path.join(root, 'scenes', x, camera, 'depth', str(img_num).zfill(4)+'.png'))
                self.labelpath.append(os.path.join(root, 'scenes', x, camera, 'label', str(img_num).zfill(4)+'.png'))
                self.metapath.append(os.path.join(root, 'scenes', x, camera, 'meta', str(img_num).zfill(4)+'.mat'))
                self.scenename.append(x.strip())
                self.frameid.append(img_num)
            if self.load_label:
                collision_labels = np.load(os.path.join(root, 'collision_label', x.strip(),  'collision_labels.npz'))
                self.collision_labels[x.strip()] = {}
                for i in range(len(collision_labels)):
                    self.collision_labels[x.strip()][i] = collision_labels['arr_{}'.format(i)]

    def scene_list(self):
        return self.scenename

    def __len__(self):
        return len(self.depthpath)

    def augment_data(self, point_clouds, object_poses_list):
        # Flipping along the YZ plane
        if np.random.random() > 0.5:
            flip_mat = np.array([[-1, 0, 0],
                                [ 0, 1, 0],
                                [ 0, 0, 1]])
            point_clouds = transform_point_cloud(point_clouds, flip_mat, '3x3')
            for i in range(len(object_poses_list)):
                object_poses_list[i] = np.dot(flip_mat, object_poses_list[i]).astype(np.float32)

        # Rotation along up-axis/Z-axis
        rot_angle = (np.random.random()*np.pi/3) - np.pi/6 # -30 ~ +30 degree
        c, s = np.cos(rot_angle), np.sin(rot_angle)
        rot_mat = np.array([[1, 0, 0],
                            [0, c,-s],
                            [0, s, c]])
        point_clouds = transform_point_cloud(point_clouds, rot_mat, '3x3')
        for i in range(len(object_poses_list)):
            object_poses_list[i] = np.dot(rot_mat, object_poses_list[i]).astype(np.float32)

        return point_clouds, object_poses_list

    def __getitem__(self, index):
        if self.load_label:
            return self.get_data_label(index)
        else:
            return self.get_data(index)
    def loadGrasp(self,index ,format = '6d', camera='realsense', grasp_labels = None, collision_labels = None, fric_coef_thresh=0.4):
        '''
        **Input:**

        - sceneId: int of scene id.

        - annId: int of annotation id.

        - format: string of grasp format, '6d' or 'rect'.

        - camera: string of camera type, 'kinect' or 'realsense'.

        - grasp_labels: dict of grasp labels. Call self.loadGraspLabels if not given.

        - collision_labels: dict of collision labels. Call self.loadCollisionLabels if not given.

        - fric_coef_thresh: float of the frcition coefficient threshold of the grasp. 

        **ATTENTION**

        the LOWER the friction coefficient is, the better the grasp is.

        **Output:**

        - If format == '6d', return a GraspGroup instance.

        - If format == 'rect', return a RectGraspGroup instance.
        '''
        sceneId = int(self.scenename[index][6:])
        annId = self.frameid[index]
        grasp_labels = self.grasp_labels
        collision_labels = self.collision_labels
        import numpy as np
        assert format == '6d' or format == 'rect', 'format must be "6d" or "rect"'
        if format == '6d':
            from graspnetAPI.utils.xmlhandler import xmlReader
            from graspnetAPI.utils.utils import get_obj_pose_list, generate_views, get_model_grasps, transform_points
            from graspnetAPI.utils.rotation import batch_viewpoint_params_to_matrix
            
            camera_poses = np.load(os.path.join(self.root,'scenes','scene_%04d' %(sceneId,),camera, 'camera_poses.npy'))
            camera_pose = camera_poses[annId]
            scene_reader = xmlReader(os.path.join(self.root,'scenes','scene_%04d' %(sceneId,),camera,'annotations','%04d.xml' %(annId,)))
            pose_vectors = scene_reader.getposevectorlist()

            obj_list,pose_list = get_obj_pose_list(camera_pose,pose_vectors)
            if grasp_labels is None:
                print('warning: grasp_labels are not given, calling self.loadGraspLabels to retrieve them')
                grasp_labels = self.loadGraspLabels(objIds = obj_list)
            if collision_labels is None:
                print('warning: collision_labels are not given, calling self.loadCollisionLabels to retrieve them')
                collision_labels = self.loadCollisionLabels(sceneId)

            num_views, num_angles, num_depths = 300, 12, 4
            template_views = generate_views(num_views)
            template_views = template_views[np.newaxis, :, np.newaxis, np.newaxis, :]
            template_views = np.tile(template_views, [1, 1, num_angles, num_depths, 1])

            collision_dump = collision_labels['scene_'+str(sceneId).zfill(4)]

            # grasp = dict()
            grasp_group = GraspGroup()
            for i, (obj_idx, trans) in enumerate(zip(obj_list, pose_list)):
                sampled_points, offsets, fric_coefs = grasp_labels[obj_idx+1]
                collision = collision_dump[i]
                point_inds = np.arange(sampled_points.shape[0])

                num_points = len(point_inds)
                target_points = sampled_points[:, np.newaxis, np.newaxis, np.newaxis, :]
                target_points = np.tile(target_points, [1, num_views, num_angles, num_depths, 1])
                views = np.tile(template_views, [num_points, 1, 1, 1, 1])
                angles = offsets[:, :, :, :, 0]
                depths = offsets[:, :, :, :, 1]
                widths = offsets[:, :, :, :, 2]

                mask1 = ((fric_coefs <= fric_coef_thresh) & (fric_coefs > 0) & ~collision)
                target_points = target_points[mask1]
                target_points = transform_points(target_points, trans)
                target_points = transform_points(target_points, np.linalg.inv(camera_pose))
                views = views[mask1]
                angles = angles[mask1]
                depths = depths[mask1]
                widths = widths[mask1]
                fric_coefs = fric_coefs[mask1]

                Rs = batch_viewpoint_params_to_matrix(-views, angles)
                Rs = np.matmul(trans[np.newaxis, :3, :3], Rs)
                Rs = np.matmul(np.linalg.inv(camera_pose)[np.newaxis,:3,:3], Rs)

                num_grasp = widths.shape[0]
                scores = (1.1 - fric_coefs).reshape(-1,1)
                widths = widths.reshape(-1,1)
                heights = GRASP_HEIGHT * np.ones((num_grasp,1))
                depths = depths.reshape(-1,1)
                rotations = Rs.reshape((-1,9))
                object_ids = obj_idx * np.ones((num_grasp,1), dtype=np.int32)

                obj_grasp_array = np.hstack([scores, widths, heights, depths, rotations, target_points, object_ids]).astype(np.float32)

                grasp_group.grasp_group_array = np.concatenate((grasp_group.grasp_group_array, obj_grasp_array))
            return grasp_group
        else:
            # 'rect'
            rect_grasps = RectGraspGroup(os.path.join(self.root,'scenes','scene_%04d' % sceneId,camera,'rect','%04d.npy' % annId))
            return rect_grasps

    def get_data(self, index, return_raw_cloud=False):
        color = np.array(Image.open(self.colorpath[index]), dtype=np.float32) / 255.0
        depth = np.array(Image.open(self.depthpath[index]))
        seg = np.array(Image.open(self.labelpath[index]))
        meta = scio.loadmat(self.metapath[index])
        scene = self.scenename[index]
        try:
            intrinsic = meta['intrinsic_matrix']
            factor_depth = meta['factor_depth']
        except Exception as e:
            print(repr(e))
            print(scene)
        camera = CameraInfo(1280.0, 720.0, intrinsic[0][0], intrinsic[1][1], intrinsic[0][2], intrinsic[1][2], factor_depth)

        # generate cloud
        cloud = create_point_cloud_from_depth_image(depth, camera, organized=True)

        # get valid points
        depth_mask = (depth > 0)
        seg_mask = (seg > 0)
        if self.remove_outlier:
            camera_poses = np.load(os.path.join(self.root, 'scenes', scene, self.camera, 'camera_poses.npy'))
            align_mat = np.load(os.path.join(self.root, 'scenes', scene, self.camera, 'cam0_wrt_table.npy'))
            trans = np.dot(align_mat, camera_poses[self.frameid[index]])
            workspace_mask = get_workspace_mask(cloud, seg, trans=trans, organized=True, outlier=0.02)
            mask = (depth_mask & workspace_mask)
        else:
            mask = depth_mask
        cloud_masked = cloud[mask]
        color_masked = color[mask]
        seg_masked = seg[mask]
        if return_raw_cloud:
            return cloud_masked, color_masked

        # sample points
        if len(cloud_masked) >= self.num_points:
            idxs = np.random.choice(len(cloud_masked), self.num_points, replace=False)
        else:
            idxs1 = np.arange(len(cloud_masked))
            idxs2 = np.random.choice(len(cloud_masked), self.num_points-len(cloud_masked), replace=True)
            idxs = np.concatenate([idxs1, idxs2], axis=0)
        cloud_sampled = cloud_masked[idxs]
        color_sampled = color_masked[idxs]
        
        ret_dict = {}
        ret_dict['point_clouds'] = cloud_sampled.astype(np.float32)
        ret_dict['cloud_colors'] = color_sampled.astype(np.float32)

        return ret_dict
    
    def get_data_label(self, index):
        color = np.array(Image.open(self.colorpath[index]), dtype=np.float32) / 255.0
        depth = np.array(Image.open(self.depthpath[index]))
        seg = np.array(Image.open(self.labelpath[index]))
        meta = scio.loadmat(self.metapath[index])
        scene = self.scenename[index]
        try:
            obj_idxs = meta['cls_indexes'].flatten().astype(np.int32)
            poses = meta['poses']
            intrinsic = meta['intrinsic_matrix']
            factor_depth = meta['factor_depth']
        except Exception as e:
            print(repr(e))
            print(scene)
        camera = CameraInfo(1280.0, 720.0, intrinsic[0][0], intrinsic[1][1], intrinsic[0][2], intrinsic[1][2], factor_depth)

        # generate cloud
        cloud = create_point_cloud_from_depth_image(depth, camera, organized=True)

        # get valid points
        depth_mask = (depth > 0)
        seg_mask = (seg > 0)
        if self.remove_outlier:
            camera_poses = np.load(os.path.join(self.root, 'scenes', scene, self.camera, 'camera_poses.npy'))
            align_mat = np.load(os.path.join(self.root, 'scenes', scene, self.camera, 'cam0_wrt_table.npy'))
            trans = np.dot(align_mat, camera_poses[self.frameid[index]])
            workspace_mask = get_workspace_mask(cloud, seg, trans=trans, organized=True, outlier=0.02)
            mask = (depth_mask & workspace_mask)
        else:
            mask = depth_mask
        cloud_masked = cloud[mask]
        color_masked = color[mask]
        seg_masked = seg[mask]

        # sample points
        if len(cloud_masked) >= self.num_points:
            idxs = np.random.choice(len(cloud_masked), self.num_points, replace=False)
        else:
            idxs1 = np.arange(len(cloud_masked))
            idxs2 = np.random.choice(len(cloud_masked), self.num_points-len(cloud_masked), replace=True)
            idxs = np.concatenate([idxs1, idxs2], axis=0)
        cloud_sampled = cloud_masked[idxs]
        color_sampled = color_masked[idxs]
#         seg_sampled = seg_masked[idxs]
#         objectness_label = seg_sampled.copy()
#         objectness_label[objectness_label>1] = 1
        
#         object_poses_list = []
#         grasp_points_list = []
#         grasp_offsets_list = []
#         grasp_scores_list = []
#         grasp_tolerance_list = []
#         for i, obj_idx in enumerate(obj_idxs):
#             if obj_idx not in self.valid_obj_idxs:
#                 continue
#             if (seg_sampled == obj_idx).sum() < 50:
#                 continue
#             object_poses_list.append(poses[:, :, i])
#             points, offsets, scores, tolerance = self.grasp_labels[obj_idx]
#             collision = self.collision_labels[scene][i] #(Np, V, A, D)

#             # remove invisible grasp points
#             if self.remove_invisible:
#                 visible_mask = remove_invisible_grasp_points(cloud_sampled[seg_sampled==obj_idx], points, poses[:,:,i], th=0.01)
#                 points = points[visible_mask]
#                 offsets = offsets[visible_mask]
#                 scores = scores[visible_mask]
#                 tolerance = tolerance[visible_mask]
#                 collision = collision[visible_mask]

#             idxs = np.random.choice(len(points), min(max(int(len(points)/4),300),len(points)), replace=False)
#             grasp_points_list.append(points[idxs])
#             grasp_offsets_list.append(offsets[idxs])
#             collision = collision[idxs].copy()
#             scores = scores[idxs].copy()
#             scores[collision] = 0
#             grasp_scores_list.append(scores)
#             tolerance = tolerance[idxs].copy()
#             tolerance[collision] = 0
#             grasp_tolerance_list.append(tolerance)
        
#         if self.augment:
#             cloud_sampled, object_poses_list = self.augment_data(cloud_sampled, object_poses_list)
        T88 = self.loadGrasp(index).nms()
#         for i in T88.__len__():
#             T88[i].
            
        ret_dict = {}
        ret_dict['point_clouds'] = cloud_sampled.astype(np.float32)
        ret_dict['cloud_colors'] = color_sampled.astype(np.float32)
#         ret_dict['objectness_label'] = objectness_label.astype(np.int64)
#         ret_dict['object_poses_list'] = object_poses_list
#         ret_dict['grasp_points_list'] = grasp_points_list
#         ret_dict['grasp_offsets_list'] = grasp_offsets_list
#         ret_dict['grasp_labels_list'] = grasp_scores_list
#         ret_dict['grasp_tolerance_list'] = grasp_tolerance_list
        ret_dict['grasp_list'] = T88

        return ret_dict

def load_grasp_labels(root):
    obj_names = list(range(88))
    valid_obj_idxs = []
    grasp_labels = {}
    for i, obj_name in enumerate(tqdm(obj_names, desc='Loading grasping labels...')):
#         if i == 18: continue
        valid_obj_idxs.append(i + 1) #here align with label png
        label = np.load(os.path.join(root, 'grasp_label', '{}_labels.npz'.format(str(i).zfill(3))))
        tolerance = np.load(os.path.join(BASE_DIR, 'tolerance', '{}_tolerance.npy'.format(str(i).zfill(3))))
        grasp_labels[i + 1] = (label['points'].astype(np.float32), label['offsets'].astype(np.float32),
                                label['scores'].astype(np.float32))#, tolerance)

    return valid_obj_idxs, grasp_labels
# def loadGraspLabels(self, objIds=None):
#         '''
#         **Input:**

#         - objIds: int or list of int of the object ids.

#         **Output:**

#         - a dict of grasplabels of each object. 
#         '''
#         # load object-level grasp labels of the given obj ids
#         objIds = self.objIds if objIds is None else objIds
#         assert _isArrayLike(objIds) or isinstance(objIds, int), 'objIds must be an integer or a list/numpy array of integers'
#         objIds = objIds if _isArrayLike(objIds) else [objIds]
#         graspLabels = {}
#         for i in tqdm(objIds, desc='Loading grasping labels...'):
#             file = np.load(os.path.join(self.root, 'grasp_label', '{}_labels.npz'.format(str(i).zfill(3))))
#             graspLabels[i] = (file['points'].astype(np.float32), file['offsets'].astype(np.float32), file['scores'].astype(np.float32))
#         return graspLabels
def collate_fn(batch):
    if type(batch[0]).__module__ == 'numpy':
        return torch.stack([torch.from_numpy(b) for b in batch], 0)
    elif isinstance(batch[0], container_abcs.Mapping):
        return {key:collate_fn([d[key] for d in batch]) for key in batch[0]}
    elif isinstance(batch[0], container_abcs.Sequence):
        return [[torch.from_numpy(sample) for sample in b] for b in batch]
    
    raise TypeError("batch must contain tensors, dicts or lists; found {}".format(type(batch[0])))
    
def distance_by_translation_point(p1, p2):
    """
      Gets two nx3 points and computes the distance between point p1 and p2.
    """
    return np.sqrt(np.sum(np.square(p1 - p2), axis=-1))
# if __name__ == "__main__":
#     root = '/home/po/TM5/graspnetAPI'
#     valid_obj_idxs, grasp_labels = load_grasp_labels(root)
#     train_dataset = GraspNetDataset(root, valid_obj_idxs, grasp_labels, split='train', remove_outlier=True, remove_invisible=True, num_points=20000)
#     print(len(train_dataset))

#     end_points = train_dataset[233]
#     cloud = end_points['point_clouds']
#     seg = end_points['objectness_label']
#     print(cloud.shape)
#     print(cloud.dtype)
#     print(cloud[:,0].min(), cloud[:,0].max())
#     print(cloud[:,1].min(), cloud[:,1].max())
#     print(cloud[:,2].min(), cloud[:,2].max())
#     print(seg.shape)
#     print((seg>0).sum())
#     print(seg.dtype)
#     print(np.unique(seg))
root = '/home/po/TM5/graspnetAPI'
valid_obj_idxs, grasp_labels = load_grasp_labels(root)
train_dataset = GraspNetDataset(root, valid_obj_idxs, grasp_labels, split='train', remove_outlier=True, remove_invisible=True, num_points=20000)
print(len(train_dataset))

Loading data path and collision labels...: 100%|██████████| 100/100 [00:18<00:00,  5.44it/s]

25600


In [5]:
from pyquaternion import Quaternion

In [6]:
def worker(index):
    index = index
    grasp_list = train_dataset.__getitem__(index)['grasp_list']
    point_clouds = train_dataset.__getitem__(index)['point_clouds']
#     template_grasp = np.zeros([10000,7],dtype=float)
    # import datetime
    # starttime = datetime.datetime.now()
    # print("The time used to execute this is given below")
    count = 0
    label_count = 0
    print(grasp_list.__len__(),'\n',len(point_clouds),'\n',count,'\n',label_count)
    for i in range(grasp_list.__len__()):
        grasp_NOi = grasp_list[i]
        xyz_NOi = grasp_NOi.translation
        q_NOi = grasp_NOi.rotation_matrix
#         print('grasp_i:',i)
        for j in range(len(point_clouds)):
            result = distance_by_translation_point(xyz_NOi,point_clouds[j])
#             print('j:',j)
            if result < 0.005 :
                try:
                    train_dataset.template_grasp[j,3:7] = Quaternion(matrix=grasp_NOi.rotation_matrix).elements
                    train_dataset.template_grasp[j,0:3] = grasp_NOi.translation
                    label_count = label_count + 1
                    print('count:',count,'\n','label_count:',label_count,'\n','i,j:',i,j,'\n\ng')
                    print('grasp_label_no:',j,'grasp_7element:',train_dataset.template_grasp[j,:])
                    break
                except:
                    print('fucki',i,'fuckj',j)
                    break
            count = count + 1
#     endtime = datetime.datetime.now()
#     print (endtime - starttime).seconds
    print(grasp_list.__len__(),'\n',len(point_clouds),'\n',count,'\n',label_count)

In [7]:
worker(0)

655 
 20000 
 0 
 0
count: 8117 
 label_count: 1 
 i,j: 0 8117 

g
grasp_label_no: 8117 grasp_7element: [-0.18646973 -0.00794195  0.45824611 -0.04898256  0.37071347 -0.56280049
  0.73717561]
count: 12941 
 label_count: 2 
 i,j: 1 4824 

g
grasp_label_no: 4824 grasp_7element: [ 0.18871266 -0.02459453  0.42188242 -0.24304258  0.57025689 -0.09607277
  0.77878587]
count: 24161 
 label_count: 3 
 i,j: 2 11220 

g
grasp_label_no: 11220 grasp_7element: [-0.22482614 -0.12943552  0.44650319  0.20036642  0.59461203 -0.26828426
  0.73096744]
count: 40213 
 label_count: 4 
 i,j: 3 16052 

g
grasp_label_no: 16052 grasp_7element: [-0.12179194  0.03839819  0.4531793   0.75788231 -0.14729426 -0.60044768
 -0.20828193]
count: 68330 
 label_count: 5 
 i,j: 5 8117 

g
grasp_label_no: 8117 grasp_7element: [-0.18646973 -0.00794195  0.45824611  0.1719397   0.24656285 -0.39822188
  0.8666388 ]
count: 88789 
 label_count: 6 
 i,j: 7 459 

g
grasp_label_no: 459 grasp_7element: [-0.13717067 -0.09576309  0.459982

count: 1125916 
 label_count: 48 
 i,j: 89 4267 

g
grasp_label_no: 4267 grasp_7element: [-0.19419333 -0.1366253   0.44086418 -0.02277333  0.60016798 -0.14023241
  0.78715607]
fucki 90 fuckj 994
count: 1140474 
 label_count: 49 
 i,j: 91 13564 

g
grasp_label_no: 13564 grasp_7element: [ 0.04506119 -0.07477719  0.40352809 -0.20100193  0.89015395  0.07455681
  0.40207644]
count: 1141468 
 label_count: 50 
 i,j: 92 994 

g
grasp_label_no: 994 grasp_7element: [-0.10948828  0.03070967  0.44898608  0.37921646  0.13099156 -0.53155437
  0.74597989]
count: 1146117 
 label_count: 51 
 i,j: 93 4649 

g
grasp_label_no: 4649 grasp_7element: [-0.10954917  0.02678119  0.45085949 -0.54824309  0.45801602  0.63489526
  0.29420892]
count: 1189009 
 label_count: 52 
 i,j: 96 2892 

g
grasp_label_no: 2892 grasp_7element: [ 0.02272156 -0.02690309  0.41976005 -0.57625782  0.27652879  0.70019054
  0.31810683]
count: 1189923 
 label_count: 53 
 i,j: 97 914 

g
grasp_label_no: 914 grasp_7element: [ 0.00823515 -

fucki 177 fuckj 935
count: 2188483 
 label_count: 95 
 i,j: 178 2237 

g
grasp_label_no: 2237 grasp_7element: [ 2.29725335e-02  7.64508382e-04  4.34527487e-01 -7.16262344e-02
 -2.03365104e-01  9.34156296e-01 -2.84366538e-01]
count: 2230542 
 label_count: 96 
 i,j: 181 2059 

g
grasp_label_no: 2059 grasp_7element: [ 0.01332121 -0.0393685   0.42293143 -0.40776499  0.57141919  0.65004555
  0.29094428]
count: 2230882 
 label_count: 97 
 i,j: 182 340 

g
grasp_label_no: 340 grasp_7element: [-0.0987878   0.0324512   0.44806197  0.58430527  0.4110189  -0.44850699
  0.53711488]
count: 2235927 
 label_count: 98 
 i,j: 183 5045 

g
grasp_label_no: 5045 grasp_7element: [ 0.04315479 -0.01400025  0.43548128 -0.19255167 -0.42510821  0.79949089
 -0.3781814 ]
count: 2297037 
 label_count: 99 
 i,j: 187 1110 

g
grasp_label_no: 1110 grasp_7element: [-0.09921697  0.0369808   0.44716519 -0.23886636  0.48218561 -0.45665097
  0.70845596]
count: 2358147 
 label_count: 100 
 i,j: 191 1110 

g
grasp_label_no:

count: 3840968 
 label_count: 142 
 i,j: 298 3230 

g
grasp_label_no: 3230 grasp_7element: [-0.17995976 -0.01233434  0.45769525  0.2874111   0.27382924 -0.06060559
  0.91582715]
count: 3864833 
 label_count: 143 
 i,j: 300 3865 

g
grasp_label_no: 3865 grasp_7element: [-2.83907384e-01 -4.67012549e-04  4.66395110e-01 -4.68455259e-02
  6.76169982e-01 -1.03378727e-01  7.27950888e-01]
count: 3889688 
 label_count: 144 
 i,j: 302 4855 

g
grasp_label_no: 4855 grasp_7element: [-0.30208266 -0.06463823  0.45530623  0.05221176  0.68935919  0.58533492
  0.42360459]
count: 3939279 
 label_count: 145 
 i,j: 305 9591 

g
grasp_label_no: 9591 grasp_7element: [-0.30910307 -0.0127178   0.45617795  0.66280784  0.59719987 -0.3583051
 -0.27505555]
count: 4006804 
 label_count: 146 
 i,j: 309 7525 

g
grasp_label_no: 7525 grasp_7element: [-0.29094118 -0.00658601  0.4623459   0.24379751  0.72642521  0.20229155
  0.60987478]
count: 4048298 
 label_count: 147 
 i,j: 312 1494 

g
grasp_label_no: 1494 grasp_7e

count: 4862475 
 label_count: 189 
 i,j: 385 420 

g
grasp_label_no: 420 grasp_7element: [-0.27170444 -0.02313952  0.46055472  0.08089545  0.89015315  0.12542639
  0.43052469]
count: 4907330 
 label_count: 190 
 i,j: 388 4855 

g
grasp_label_no: 4855 grasp_7element: [-0.30121657 -0.06388725  0.45579904  0.46414398  0.27457028  0.20897515
  0.81578852]
count: 4940383 
 label_count: 191 
 i,j: 390 13053 

g
grasp_label_no: 13053 grasp_7element: [-0.17679727 -0.13710503  0.44448546 -0.13543762  0.48667647  0.05653687
  0.86116565]
count: 4941382 
 label_count: 192 
 i,j: 391 999 

g
grasp_label_no: 999 grasp_7element: [-2.94781417e-01  8.30748177e-04  4.70442653e-01  8.79920112e-01
  2.60203755e-01  1.47070032e-03 -3.97532979e-01]
count: 5023537 
 label_count: 193 
 i,j: 396 2155 

g
grasp_label_no: 2155 grasp_7element: [-0.25793919 -0.03426027  0.44739911  0.08550776  0.70644283  0.07436055
  0.69863972]
count: 5024308 
 label_count: 194 
 i,j: 397 771 

g
grasp_label_no: 771 grasp_7elem

count: 7251736 
 label_count: 236 
 i,j: 539 17884 

g
grasp_label_no: 17884 grasp_7element: [-0.24258521 -0.06882975  0.46020335 -0.38342637  0.46228174  0.36793638
  0.70986098]
count: 7253278 
 label_count: 237 
 i,j: 540 1542 

g
grasp_label_no: 1542 grasp_7element: [ 0.12464272 -0.03858849  0.40546098 -0.09741849  0.4118231  -0.21359514
  0.88050466]
count: 7295050 
 label_count: 238 
 i,j: 543 1772 

g
grasp_label_no: 1772 grasp_7element: [-0.00586256 -0.09352414  0.41750425 -0.30426797  0.92774465  0.20919989
  0.05427959]
count: 7315256 
 label_count: 239 
 i,j: 545 206 

g
grasp_label_no: 206 grasp_7element: [-0.16235474 -0.1106296   0.44850034  0.37870538  0.26896665 -0.23707806
  0.8532486 ]
count: 7362197 
 label_count: 240 
 i,j: 548 6941 

g
grasp_label_no: 6941 grasp_7element: [-0.1970152  -0.12108846  0.44581357 -0.00599523  0.37011966 -0.36835943
  0.85281113]
count: 7404094 
 label_count: 241 
 i,j: 551 1897 

g
grasp_label_no: 1897 grasp_7element: [-0.17311139 -0.115

count: 8181350 
 label_count: 285 
 i,j: 626 18712 

g
grasp_label_no: 18712 grasp_7element: [-0.24444425 -0.03822276  0.45229411  0.41297006  0.26535359 -0.10041726
  0.86542452]
count: 8216075 
 label_count: 286 
 i,j: 628 14725 

g
grasp_label_no: 14725 grasp_7element: [-0.19547372 -0.04464782  0.45325658 -0.25334629  0.2238534   0.26001539
  0.90448731]
count: 8223709 
 label_count: 287 
 i,j: 629 7634 

g
grasp_label_no: 7634 grasp_7element: [-0.2072289  -0.05032461  0.45367807  0.07060362 -0.03551472  0.25968563
  0.96245375]
count: 8231343 
 label_count: 288 
 i,j: 630 7634 

g
grasp_label_no: 7634 grasp_7element: [-0.2072289  -0.05032461  0.45367807  0.35642886  0.78349688  0.21630186
  0.46076525]
count: 8461070 
 label_count: 289 
 i,j: 642 9727 

g
grasp_label_no: 9727 grasp_7element: [-0.29895726 -0.00648732  0.46371585  0.80161372  0.24566015 -0.24133832
 -0.48869453]
count: 8476659 
 label_count: 290 
 i,j: 643 15589 

g
grasp_label_no: 15589 grasp_7element: [-0.31930688 

In [ ]:
grasp_list = train_dataset.__getitem__(1)['grasp_list']
point_clouds = train_dataset.__getitem__(1)['point_clouds']
cloud_colors = train_dataset.__getitem__(1)['cloud_colors']

In [ ]:
template_grasp = np.zeros([10000,7],dtype=float)

In [ ]:
train_dataset.template_grasp.shape

In [ ]:
grasp_list[0].translation

In [ ]:
obj_names = list(range(88))
valid_obj_idxs = []
grasp_labels = {}
for i, obj_name in enumerate(tqdm(obj_names, desc='Loading grasping labels...')):
#         if i == 18: continue
    valid_obj_idxs.append(i + 1) #here align with label png
    label = np.load(os.path.join(root, 'grasp_label', '{}_labels.npz'.format(str(i).zfill(3))))
    tolerance = np.load(os.path.join(BASE_DIR, 'tolerance', '{}_tolerance.npy'.format(str(i).zfill(3))))
    grasp_labels[i + 1] = (label['points'].astype(np.float32), label['offsets'].astype(np.float32),
                            label['scores'].astype(np.float32))#, tolerance)

In [ ]:
grasp_labels

In [ ]:
A = Quaternion(matrix=grasp_list[0].rotation_matrix)
A.elements

In [ ]:
index = 0
template_grasp[index,0:3] = grasp_list[0].translation
template_grasp[index,3:7] = Quaternion(matrix=grasp_list[0].rotation_matrix).elements
template_grasp[index,3:7]

In [ ]:
point_clouds.shape

In [ ]:
template_grasp = np.zeros([20000,7])
worker(1)

In [ ]:
pre_dataset = np.zeros([25600,])

In [ ]:
train_dataset.template_grasp[5616,:]

In [ ]:
len(train_dataset.frameid)

In [ ]:
TRAIN_DATALOADER = DataLoader(train_dataset, batch_size=2, shuffle=True,
    num_workers=4, worker_init_fn=my_worker_init_fn, collate_fn=collate_fn)

In [ ]:
distance_by_translation_point(grasp_list[652].translation,point_clouds[653])

In [ ]:
grasp_rotation = TT87[651].rotation_matrix
grasp_translation = TT87[651].translation
print(grasp_rotation,'\n=======================================\n',grasp_translation)

In [ ]:
import datetime
starttime = datetime.datetime.now()
print("The time used to execute this is given below")
count = 0
for i in range(grasp_list.__len__()):#points = 20000
    grasp_NOi = grasp_list[i]
    xyz_NOi = grasp_NOi.translation
    q_NOi = grasp_NOi.rotation_matrix
    for j in range(len(point_clouds)):
        result = distance_by_translation_point(xyz_NOi,point_clouds[j])
        print(result)
        count = count + 1
endtime = datetime.datetime.now()
print (endtime - starttime).seconds
print(grasp_list.__len__(),len(point_clouds),count)

In [ ]:
rotation = np.eye(3)
q8d1 = Quaternion(matrix=g_r) # Using 3x3 rotation matrix
q8d1

In [ ]:
A = np.array([1,1,1])
B = np.array([1,1,2])

In [ ]:
distance_by_translation_point(TT88[651],grasp_translation)

In [ ]:
def distance_by_translation_point(p1, p2):
    """
      Gets two nx3 points and computes the distance between point p1 and p2.
    """
    return np.sqrt(np.sum(np.square(p1 - p2), axis=-1))

In [ ]:
import datetime
starttime = datetime.datetime.now()
print("The time used to execute this is given below")
for i in range(25600):
    g_l = len(train_dataset.__getitem__(i)['grasp_list'])
    p_c = len(train_dataset.__getitem__(i)['point_clouds'])
    print('NUM:',i,'grasp_list:',g_l,'point_clouds:',p_c)
    print(datetime.datetime.now())
endtime = datetime.datetime.now()
print (endtime - starttime).seconds

In [ ]:
import time
start = time.time()
print("The time used to execute this is given below")

for i in range(500):
    train_dataset.__getitem__(i)
    print(time.time())
end = time.time()

print(end - start)

In [ ]:
""" Training routine for GraspNet baseline model. """

import os
import sys
import numpy as np
from datetime import datetime
import argparse

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

In [ ]:
# Init datasets and dataloaders 
def my_worker_init_fn(worker_id):
    np.random.seed(np.random.get_state()[1][0] + worker_id)
    pass

In [ ]:
TRAIN_DATALOADER = DataLoader(train_dataset, batch_size=2, shuffle=True,
    num_workers=4, worker_init_fn=my_worker_init_fn, collate_fn=collate_fn)

In [ ]:
fuck = next(iter(TRAIN_DATALOADER))

In [ ]:
for batch_idx, batch_data_label in enumerate(TRAIN_DATALOADER):
    print( batch_idx,batch_data_label)

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

for batch_idx, batch_data_label in enumerate(TRAIN_DATALOADER):
        for key in batch_data_label:
            if 'list' in key:
                for i in range(len(batch_data_label[key])):
                    for j in range(len(batch_data_label[key][i])):
                        batch_data_label[key][i][j] = batch_data_label[key][i][j].to(device)
            else:
                batch_data_label[key] = batch_data_label[key].to(device)